In [201]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf 
from tensorflow import keras
from keras import Input
from keras import layers
from keras.optimizers import RMSprop
from keras import backend as K
from scipy.spatial import distance

def ComputeMaxDistance(X1, X2):
    max_dist = 0
    for x1 in X1:
        for x2 in X2:
            dist = distance.euclidean(x1, x2)
            max_dist = max(dist, max_dist)
    return max_dist

def RBF_kernel(x, centers, sigma):
  # res = []
  # for c in centers:
    # res.append(math.exp((-np.linalg.norm(x - c)**2) / (2*sigma**2)))
  # res = [math.exp(-np.linalg.norm(x - c)**2) / (2 * sigma**2) for c in centers]
  res = [math.exp(-np.linalg.norm(x - c)**2 / (2 * sigma**2)) for c in centers]
  return res

def RBF_layer_output(x, centers, sigma):
  # res = []
  # for x_ in x: 
  #   res.append(RBF_kernel(x_, centers, sigma))
  # res = [RBF_kernel(x_, centers, sigma) for x_ in x]
  # res = [0 for x_ in x]
  res = [RBF_kernel(x_, centers, sigma) for x_ in x]

  return res

class RBF(keras.layers.Layer):

  def __init__(self, num_units):
    self.num_units = num_units
    super(RBF, self).__init__()

  def build(self, input_shape):
    # shape1 = tf.TensorShape((input_shape[1], self.num_units))
    # shape2 = tf.TensorShape((1, self.num_units))
    self.centers = self.add_weight(shape=(input_shape[1], self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    self.sigma = self.add_weight(shape=(1, self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    # self.betas = np.ones(self.num_units) / (2 * (sigma**2))
    # self.sigma    = self.add_weight(shape=1, initializer=tf.initializers.RandomNormal, trainable=False)
    super(RBF, self).build(input_shape)

  def call(self, inputs):
    X_ = tf.expand_dims(inputs, axis=1)
    C_ = tf.expand_dims(tf.transpose(self.centers), axis=0)
    D_ = X_ - C_
    return tf.exp(-tf.norm(D_, axis=2)**2) / (2 * self.sigma[0]**2) # TODO change sigma[0]

  def get_config(self):
    base_config = super(RBF, self).get.config
    base_config['num_units'] = self.num_units
    return base_config

  def compute_params(self, inputs):
    kmeans = KMeans(n_clusters=self.num_units, random_state=0, copy_x=True).fit(inputs)
    self.centers = kmeans.cluster_centers_
    d_max = ComputeMaxDistance(self.centers, self.centers)
    self.sigma = d_max / (math.sqrt(2 * self.centers.shape[0]))


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
from keras import Input
from keras import layers
from keras.optimizers import RMSprop
import utils
import rbf_layer

# ---------------------------------------------------------------------------- #
#                                   read data                                  #
# ---------------------------------------------------------------------------- #

data = "http://lib.stat.cmu.edu/datasets/boston"
# data = "boston.csv"
raw_df = pd.read_csv(data, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# ---------------------------------------------------------------------------- #
#                                normalize data                                #
# ---------------------------------------------------------------------------- #

preprocessing.scale(X, copy=False)
preprocessing.scale(y, copy=False)

# ---------------------------------------------------------------------------- #
#                      split data to training and testing                      #
# ---------------------------------------------------------------------------- #

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

n_train = X_train.shape[0]
hidden_size = int(0.1 * n_train)

In [211]:
rbf_layer_ = RBF(hidden_size)
rbf_layer_.compute_params(X_train)
# rbf_layer_.compute_params(X_train)

from keras import layers

model = keras.Sequential()
model.add(Input(shape=(13,)))
# model.add(rbf_layer_)
# model.add(layers.Dense(12, input_shape=(13,), activation='relu'))
model.add(rbf_layer_)
model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


Tensor("rbf_36/ExpandDims:0", shape=(None, 1, 13), dtype=float32)
Tensor("rbf_36/ExpandDims_1:0", shape=(1, 37, 13), dtype=float32)
Tensor("rbf_36/sub:0", shape=(None, 37, 13), dtype=float32)
Tensor("rbf_36/sub_1:0", shape=(37, 13), dtype=float32)


In [219]:
# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
#               metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=10)



Epoch 1/5
Tensor("sequential_36/rbf_36/ExpandDims:0", shape=(None, 1, 13), dtype=float32)
Tensor("sequential_36/rbf_36/ExpandDims_1:0", shape=(1, 37, 13), dtype=float32)
Tensor("sequential_36/rbf_36/sub:0", shape=(None, 37, 13), dtype=float32)
Tensor("sequential_36/rbf_36/sub_1:0", shape=(37, 13), dtype=float32)
Tensor("sequential_36/rbf_36/ExpandDims:0", shape=(None, 1, 13), dtype=float32)
Tensor("sequential_36/rbf_36/ExpandDims_1:0", shape=(1, 37, 13), dtype=float32)
Tensor("sequential_36/rbf_36/sub:0", shape=(None, 37, 13), dtype=float32)
Tensor("sequential_36/rbf_36/sub_1:0", shape=(37, 13), dtype=float32)
38/38 [==============================] - 1s 2ms/step - loss: 0.0111 - accuracy: 0.0000e+00
Epoch 2/5
38/38 [==============================] - 0s 2ms/step - loss: -0.0073 - accuracy: 0.0000e+00
Epoch 3/5
38/38 [==============================] - 0s 2ms/step - loss: -0.0354 - accuracy: 0.0000e+00
Epoch 4/5
38/38 [==============================] - 0s 2ms/step - loss: -0.0339 - accura

In [220]:
print(history.params)
model.summary()
# y = model.predict(X_train)

# print(y[2])
# a = tf.constant([1, 2])
# print(type(a))


# print(y)

# print(norm(y[0], 1))
# print(norm(y[0][0], 2))
# print(norm(y[0][36],2))

{'verbose': 1, 'epochs': 5, 'steps': 38}
Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rbf_36 (RBF)                (None, 37)                518       
                                                                 
 dense_4 (Dense)             (None, 8)                 304       
                                                                 
Total params: 822
Trainable params: 304
Non-trainable params: 518
_________________________________________________________________


In [95]:

# x = tf.constant([1, 2, 3])

x = tf.constant([[1, 2, 3], [4, 5, 6]])
x = tf.expand_dims(x, axis=1)

centers = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
centers = tf.expand_dims(centers, axis=0)

print(x)
print(centers)
print(x - centers)

# res = []

# for x_ in x:
#     res.append(x_-centers)

# # print(res)

# # result = [res_ for res_ in res]
# # print(res.shape)

# # for res_ in res:
# #     print(res_)

# # tf.concat([res[0], res[1]], 0)
# # rrrr = tf.expand_dims(res, 0)
# rrrr = tf.convert_to_tensor(res)
# print(rrrr)

tf.Tensor(
[[[1 2 3]]

 [[4 5 6]]], shape=(2, 1, 3), dtype=int32)
tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]
  [10 11 12]]], shape=(1, 4, 3), dtype=int32)
tf.Tensor(
[[[ 0  0  0]
  [-3 -3 -3]
  [-6 -6 -6]
  [-9 -9 -9]]

 [[ 3  3  3]
  [ 0  0  0]
  [-3 -3 -3]
  [-6 -6 -6]]], shape=(2, 4, 3), dtype=int32)
